In [1]:
from Models import *
from Utils import *
from FastModels import *
import warnings
from livelossplot import PlotLossesKeras
# warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [25]:
df = pd.read_csv("/data/honzha/test/agi_vnext2/JulyFlowerIdeal_out.txt", sep="\t", header=0, names=['score', 'q', 'd', 'label'])

In [27]:
from gensim.models import KeyedVectors
import sentencepiece as spm

max_len = 15
enablePadding = True

sp = spm.SentencePieceProcessor()
sp.Load('/work/data/bpe/en.wiki.bpe.op50000.model')
bpe = KeyedVectors.load_word2vec_format("/work/data/bpe/en.wiki.bpe.op50000.d200.w2v.bin", binary=True)
bpe.index2word = [''] + bpe.index2word + ['<sos>'] + ['<eos>']  # add empty string 
nb_words = len(bpe.index2word)
# word2index
bpe_dict = {bpe.index2word[i]: i for i in range(len(bpe.index2word))}
# construct embedding_matrix
embedding_matrix = np.concatenate([np.zeros((1, bpe.vector_size)), bpe.vectors, np.zeros((2, bpe.vector_size))]) # add zero vector for empty string (i.e. used for padding)

embedding_layer = Embedding(nb_words,
                    embedding_matrix.shape[-1],
                    weights=[embedding_matrix],
                    input_length=max_len,
                    trainable=True)

In [103]:
run = BinaryClassifier(300, 128, nb_words, max_len, embedding_matrix, optimizer=Adam(), enableLSTM=True)


In [111]:
a = run.encoder.predict(x)

In [122]:
run.encoder.predict(x).flatten()

array([ 0.18763357,  0.1041089 , -0.00574999, ...,  0.02224936,
        0.13253956, -0.03529469], dtype=float32)

In [163]:
a = np.matmul(run.encoder.predict(x), run.encoder.predict(x).T)

In [169]:
a

array([[33.076576, 31.544693, 30.20147 , 29.450085],
       [31.544693, 39.099506, 33.873688, 31.947266],
       [30.20147 , 33.873688, 37.412727, 31.799318],
       [29.450085, 31.947266, 31.799318, 33.978188]], dtype=float32)

In [178]:
a.shape[0]

4

In [183]:
idx = []
mul = np.argsort(a)
for i in range(a.shape[0]):
    idx.append(mul[i][-1] if mul[i][-1] != i else mul[i][-2])
print(idx)

[1, 2, 1, 1]


In [165]:
np.argmax(a, axis=-1)

array([0, 1, 2, 3])

In [166]:
np.argsort(a)[:,-2]

array([1, 2, 1, 1])

In [160]:
x

array([[41330, 35906, 12272, 25160, 14945, 11931, 48737, 16490, 23305,
         2117,  6223,  5714, 29102,  9429, 18672],
       [37931, 35373, 24169, 48138, 32376, 34040, 47145, 36399, 35881,
        38867, 29953, 23335, 34210, 27985,  4234],
       [21541, 18240,  9829, 46909, 46836,  1934,  7101, 35773, 39227,
        13905, 42230, 39032, 34761, 35023, 40425],
       [11490,  7412, 13850, 44105, 31120, 23094, 47407, 43115,  6013,
         7820,  7379, 45588, 46723, 19922,  9155]])

In [152]:
x = np.random.randint(50005, size=(4,15))
run.model.predict([x, x]).flatten()

array([0.5554409 , 0.5815033 , 0.57286173, 0.58253545], dtype=float32)

In [148]:
for i in range(0, 1000000, 256):
    if i % 256000 == 0:
        print("here")

here
here
here
here


In [48]:
def get_agi_test_data(filename):

    if "May" in filename or "June" in filename:
        # if "/work/" in path:
        # file_dir = '/data/t-mipha/data/query_similarity_ndcg/%sIdeal.txt' % filename
        file_dir = '/data/honzha/test/agi_vnext2/%s' % (filename)
        df = pd.read_csv(file_dir, names=["score", "market", "qid", "q", "label", "d", "date"], sep="\t", header=0, error_bad_lines=False)
        df = df.dropna()
        y = np.array([0 if i == "Bad" else 1 if i == "Fare" else 2 for i in df.label.tolist()])

        qrel = convert_2_trec(df.q.tolist(), df.d.tolist(), y, True)

    elif "July" in filename:
        file_dir = '/data/honzha/test/agi_vnext2/%s' % (filename)
        # file_dir = '/data/chzho/deepqts/test_data/julyflower/julyflower_original.tsv'
        df = pd.read_csv(file_dir, names=["score", "q", "d", "label"], sep="\t", header=0, error_bad_lines=False)
        df = df.dropna()
        qrel = df.label.values
    elif "vnext" in filename:
        file_dir = '/data/honzha/test/agi_vnext2/%s' % (filename)
        # file_dir = '/data/chzho/deepqts/test_data/julyflower/julyflower_original.tsv'
        df = pd.read_csv(file_dir, names=["score", "q", "d", "mode", "label"], sep="\t", header=0, error_bad_lines=False)
        df = df.dropna()
        qrel = df.label.values
    elif "sts" in filename:

        file_dir = '/data/honzha/test/agi_vnext2/%s' % (filename)
        df = pd.read_csv(file_dir, sep="\t",encoding='utf-8',header=0,quoting=3, error_bad_lines=False, names=["score", "genre", "file", "year", "id", "label","q","d"])
        df = df.dropna()
        qrel = df.label.values

    elif "quora" in filename or "Para" in filename:
        file_dir = '/data/honzha/test/agi_vnext2/%s' % (filename)
        df = pd.read_csv(file_dir, sep="\t",encoding='utf-8',quoting=3,  header=0, error_bad_lines=False, names=["score","q","d","label", "hashid"])
        df = df.dropna()
        qrel = df.label.values

    df.q = df.q.astype(str).str.lower()
    df.d = df.d.astype(str).str.lower()

    return df, qrel

In [49]:
# =================================== Get testing data ==============================================
path = "/work/"
test_set = []
datasets = ["flower_vnext_out.txt", "MayFlowerIdeal_out.txt", "sts-dev-strip_out.txt", "JulyFlowerIdeal_out.txt", "ParaphraseIdeal_out.txt", "sts-test-strip_out.txt", "JuneFlowerIdeal_out.txt", "quora_test_out.txt"]
for i in datasets:
    print(i)
    df, qrel = get_agi_test_data(i)
    q_ = parse_texts_bpe(df.q.tolist(), sp, bpe_dict, max_len, enablePadding)
    d_ = parse_texts_bpe(df.d.tolist(), sp, bpe_dict, max_len, enablePadding)
    test_set.append([q_, d_, qrel, df, i])

flower_vnext_out.txt
MayFlowerIdeal_out.txt
sts-dev-strip_out.txt
JulyFlowerIdeal_out.txt
ParaphraseIdeal_out.txt
sts-test-strip_out.txt
JuneFlowerIdeal_out.txt


b'Skipping line 19898: expected 7 fields, saw 9\nSkipping line 20620: expected 7 fields, saw 9\nSkipping line 38039: expected 7 fields, saw 9\n'


quora_test_out.txt


In [101]:
def final_evaluate(run=None, alpha=0):
    flower_auc, may_ndcg, june_ndcg, july_auc, quora_auc, para_auc, sts_dev_pcc, sts_test_pcc = 0, 0, 0, 0, 0, 0, 0, 0
    for q, d, qrel, df, test_data in test_set:
        if run == None:
            pred = df.score.values * 0.05
        else:
            q_ = run.predict(q, batch_size=1024)
            d_ = run.predict(d, batch_size=1024)
            cosine = CosineSim(q_.shape[-1])
            pred = cosine.model.predict([q_, d_]).flatten()
            agi_pred = df.score.values
            pred = alpha * agi_pred + (1-alpha) * pred

        
        if test_data in ["MayFlowerIdeal_out.txt", "JuneFlowerIdeal_out.txt"]:
            pred = convert_2_trec(df.q.tolist(), df.d.tolist(), pred, False)
            ndcg_score, map_score = ranking_measure(qrel, pred)

        if test_data == "MayFlowerIdeal_out.txt":
            may_ndcg = ndcg_score
        elif test_data == "JuneFlowerIdeal_out.txt":
            june_ndcg = ndcg_score
        elif test_data == "JulyFlowerIdeal_out.txt":
            july_auc = auc(qrel, pred)
        elif test_data == "flower_vnext_out.txt":
            flower_auc = auc(qrel, pred)
        elif test_data ==  "ParaphraseIdeal_out.txt":
            para_auc = auc(qrel, pred)
        elif test_data == "quora_test_out.txt":
            quora_auc = auc(qrel, pred)
        elif test_data in ["sts-dev-strip_out.txt"]:
            sts_dev_pcc = scipy.stats.pearsonr(pred, qrel)[0]
        elif test_data in ["sts-test-strip_out.txt"]:
            sts_test_pcc = scipy.stats.pearsonr(pred, qrel)[0]
    return "%.4f,%.4f,%.4f,%.4f,%.4f,%.4f,%.4f,%.4f" % (flower_auc, may_ndcg, june_ndcg, july_auc, quora_auc, para_auc, sts_dev_pcc, sts_test_pcc)

In [93]:
final_evaluate()

'0.8483,0.5222,0.8532,0.6702,0.9231,0.9880,0.8188,0.7735'

In [100]:
run = load_model("/work/data/models/week10/dssm_aae_s_m2_10M_QQ_ml15_limit1000000_2018_08_26_20:20:25.encoder.h5")

In [102]:
for i in range(11):
    print(final_evaluate(run, i * 0.1))

0.7259,0.4984,0.8330,0.5604,0.7304,0.8867,0.5970,0.5456
0.7473,0.5024,0.8370,0.5726,0.7587,0.9110,0.6422,0.5879
0.7681,0.5055,0.8398,0.5852,0.7863,0.9310,0.6839,0.6287
0.7876,0.5082,0.8423,0.5979,0.8131,0.9475,0.7209,0.6666
0.8050,0.5113,0.8443,0.6110,0.8381,0.9603,0.7523,0.7003
0.8198,0.5150,0.8469,0.6236,0.8604,0.9700,0.7776,0.7284
0.8316,0.5171,0.8495,0.6352,0.8796,0.9770,0.7965,0.7502
0.8402,0.5188,0.8508,0.6461,0.8954,0.9819,0.8095,0.7653
0.8457,0.5206,0.8520,0.6562,0.9080,0.9852,0.8170,0.7737
0.8483,0.5217,0.8531,0.6668,0.9172,0.9871,0.8198,0.7762
0.8483,0.5222,0.8532,0.6702,0.9231,0.9880,0.8188,0.7735
